# Generalizations of Max-Flow

The purpose of this notebook is to learn about the min-cost flow problem, a generalization of max-flow, and to familiarize myself with implementing and solving linear programs.

## Min-Cost Flow

Recall that a flow network with demands consists of a directed graph $G = (V, E)$, where each edge $(i,j) \in E$ has a positive integer capacity $c_{ij}$ and each node $i \in V$ has an integer demand $d_i$. In a *min-cost flow* problem, each edge $(i,j) \in E$ also has a cost (or weight) $w_{ij}$. (Note that this input generalizes the input to two important problems: in max flow, the edge weights were not important while in shortest paths, the edge capacities were not important.) 

Given a flow network with capacities and costs, the goal is to find a *feasible* flow $f: E \rightarrow R^+$ --that is, a flow satisfying edge capacity constraints and node demands-- that minimizes the total cost of the flow. Explicitly, the problem can be formulated as a linear program.

### Part 1

Working with some standard benchmark instances for min-cost flow found [here](http://elib.zib.de/pub/mp-testdata/mincost/gte/index.html). The format of the data is described in the [Info](http://elib.zib.de/pub/mp-testdata/mincost/gte/info) file.Note that the data sometimes lists multiple edges between the same nodes, but with different costs or capacities.

In [1]:
import networkx as nx

def create_graph(infile):
    """Creates a directed graph as specified by the input file. Edges are annotated with 'capacity'
    and 'weight' attributes, and nodes are annotated with 'demand' attributes.
    
    Args:
        infile: the input file using the format to specify a min-cost flow problem.
        
    Returns:
        A directed graph (but not a multi-graph) with edges annotated with 'capacity' and 'weight' attributes
        and nodes annotated with 'demand' attributes.
    """
    # TODO: implement function
    #initial directed graph
    G=nx.DiGraph()
    f= open(infile,'r')
    for line in f.readlines():
        t=line[0]
        if t=='p':
            part=line.strip().split(" ")
            nn=int(part[2])
            ne=int(part[3])
        if t=='n':
            part=line.strip().split(" ")
            node=part[1]
            demand=int(part[2])
            G.add_node(node,demand=demand)
        if t=='a':
            part=line.strip().split(" ")
            tail=part[1]
            head=part[2]
            low_cap=int(part[3])
            capacity=int(part[4])
            cost=int(part[5])
            if tail not in G.nodes():
                G.add_node(tail,demand=0)
            if head not in G.nodes():
                G.add_node(head,demand=0)
            if (head,tail) in G.edges():
                nn=nn+1
                G.add_node(str(nn),demand=0)
                G.add_edge(head,str(nn),weight=cost,capacity=capacity)
                G.add_edge(str(nn),tail,weight=0,capacity=capacity)
                ne+=2
            if (head,tail) not in G.edges():
                G.add_edge(head,tail,weight=cost,capacity=capacity)
    return G
    pass

The following will check that your code outputs the expected min cost flow values on several test instances.

In [2]:
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print "Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858
print "Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788
print "Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


## Linear Programming

Instead of using special-purpose min-cost flow solvers, I am formulating the problems as linear programs and using general-purpose LP solvers to find the solutions.

In [3]:
import pulp 

def lp_flow_value(G):
    """Computes the value of the minimum cost flow by formulating and solving
    the problem as an LP.
    
    Args:
        G: a directed graph with edges annotated with 'capacity' and 'weight'
            attrbutes, and nodes annotated with 'demand' attributes.
            
    Returns:
        The value of the minimum cost flow.
    """
    # TODO: implement function
    lp_problem=pulp.LpProblem("Minimum Cost Flow", pulp.LpMinimize)
    demand=[]
    capacity=[]
    weight=[]
    total_cost=0
    demand = nx.get_node_attributes(G, 'demand')
    capacity = nx.get_edge_attributes(G, 'capacity')
    weight = nx.get_edge_attributes(G, 'weight')
    flow = pulp.LpVariable.dicts('Flow on', indexs=G.edges(), lowBound=0, upBound=None, cat=pulp.LpInteger)
    for edge in G.edges():
        total_cost=total_cost+flow[edge]*weight[edge]
    lp_problem += total_cost, "Total Cost of Flow"
    for edge in G.edges():
        lp_problem += flow[edge] <= capacity[edge]
    nodes=G.nodes()
    inflow=0
    outflow=0
    for node in G.nodes():
        inflow=0
        outflow=0
        for edge in G.in_edges(node):
            inflow=inflow+flow[edge]
        for edges in G.out_edges(node):
            outflow=outflow+flow[edges]
        lp_problem+= inflow-outflow ==demand[node]
        
    lp_problem.solve()
    min_cost=pulp.value(lp_problem.objective)
    return int(min_cost)
    pass

The following will check that the LP finds the same optimal values as previously.

In [4]:
print "Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858
print "Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788
print "Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
